In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.listdir("/content/drive/MyDrive/Colab_Notebooks"))

# Any results you write to the current directory are saved as output.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Untitled0.ipynb', 'Copy of Untitled0.ipynb', 'Untitled2.ipynb', 'Untitled3.ipynb', 'Untitled6.ipynb', 'Copy of Untitled4.ipynb', 'LSTMimdb.ipynb', 'Untitled5.ipynb', 'BERTproductreview (1).ipynb', 'Untitled4.ipynb', 'Copy of FasterRCNN_Tutorial_MeyerA.ipynb', 'Untitled7.ipynb', 'RCNN.ipynb', 'Copy of RCNN.ipynb', 'VOC2012', 'VOC2012.zip', 'VOCfolder', '.ipynb_checkpoints', 'FRCNN.ipynb', 'yelp_review_polarity_csv.zip', 'test_1copy.csv', 'test.csv', 'train.csv', 'yelp_review_polarity_csv', 'deceptive-opinion.csv', 'EDAonYelpReviews.ipynb', 'glove.6B.100d.txt', 'BERTproductreview.ipynb', 'Cnnonfashionmnsit.ipynb', 'train_1copy.csv', 'IMDB_Dataset.csv', 'IMDB Dataset.csv', 'Attention-bidirectional_glove.ipynb']


In [ ]:
!pip install tensorflow-io

In [ ]:
import tensorflow.io as tfio

In [ ]:
def my_laplacian(input, ksize, mode=None, constant_values=None, name=None):
    """
    Apply Laplacian filter to image.
    Args:
      input: A 4-D (`[N, H, W, C]`) Tensor.
      ksize: A scalar Tensor. Kernel size.
      mode: A `string`. One of "CONSTANT", "REFLECT", or "SYMMETRIC"
        (case-insensitive). Default "CONSTANT".
      constant_values: A `scalar`, the pad value to use in "CONSTANT"
        padding mode. Must be same type as input. Default 0.
      name: A name for the operation (optional).
    Returns:
      A 4-D (`[N, H, W, C]`) Tensor.
    """

    input = tf.convert_to_tensor(input)
    ksize = tf.convert_to_tensor(ksize)

    tf.debugging.assert_none_equal(tf.math.mod(ksize, 2), 0)

    ksize = tf.broadcast_to(ksize, [2])

    total = ksize[0] * ksize[1]
    index = tf.reshape(tf.range(total), ksize)
    g = tf.where(
        tf.math.equal(index, tf.math.floordiv(total - 1, 2)),
        tf.cast(1 - total, input.dtype),
        tf.cast(1, input.dtype),
    )

    input = pad(input, ksize, mode, constant_values)

    channel = tf.shape(input)[-1]
    shape = tf.concat([ksize, tf.constant([1, 1], ksize.dtype)], axis=0)
    g = tf.reshape(g, shape)
    shape = tf.concat([ksize, [channel], tf.constant([1], ksize.dtype)], axis=0)
    g = tf.broadcast_to(g, shape)
    return tf.nn.depthwise_conv2d(input, g, [1, 1, 1, 1], padding="VALID")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/IMDB_Dataset.csv')
#df = pd.read_csv('./deceptive-opinion.csv')
df.head()

#df = df.drop(["hotel", "polarity","source"], axis=1)

df.head()

df1=df.sample(frac=1)

df1.head()

from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df1['deceptive']= label_encoder.fit_transform(df1['deceptive']) 

df1['deceptive'].unique() 

df1.head()

df1.describe()

X = df1.text
y = df1.deceptive

# Some preprocesssing that will be common to all the text classification methods

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

df1.head()

import nltk
nltk.download('stopwords')


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_data(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text
    
df1['text'] = df1['text'].apply(clean_data)

X = df1.text
y = df1.deceptive

X = X.map(lambda a: clean_data(a))

from __future__ import print_function
from keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
from bs4 import BeautifulSoup
import logging
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, MaxPooling1D, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras import initializers, regularizers, constraints

train, test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)

train, val, y_train, y_val = train_test_split(train,y_train,stratify=y_train,random_state=42)

#train, val, y_train, y_val = train_test_split(X,y,stratify=y,random_state=42)

tokenizer = Tokenizer(num_words=None,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ',char_level=False)

tokenizer.fit_on_texts(X)

x_train = tokenizer.texts_to_sequences(train)

x_test = tokenizer.texts_to_sequences(test)

x_val = tokenizer.texts_to_sequences(val)

word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(X)

vocab_size = len(word_index)
print('Vocab size: {}'.format(vocab_size))
longest = max(len(seq) for seq in X)
print("Longest comment size: {}".format(longest))
average = np.mean([len(seq) for seq in X])
print("Average comment size: {}".format(average))
stdev = np.std([len(seq) for seq in X])
print("Stdev of comment size: {}".format(stdev))
max_len = int(average + stdev * 3)
print('Max comment size: {}'.format(max_len))

from keras.preprocessing.sequence import pad_sequences

processed_post_x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')
processed_post_x_test = pad_sequences(x_test, maxlen=max_len, padding='post', truncating='post')

processed_post_x_val = pad_sequences(x_val, maxlen=max_len, padding='post', truncating='post')

processed_x_train = pad_sequences(x_train, maxlen=max_len)
processed_x_test = pad_sequences(x_test, maxlen=max_len)
processed_x_val = pad_sequences(x_val, maxlen=max_len)

processed_pre_x_train = pad_sequences(x_train, maxlen=max_len)
processed_pre_x_test = pad_sequences(x_test, maxlen=max_len)
processed_pre_x_val = pad_sequences(x_val, maxlen=max_len)


print('x_train shape:', processed_x_train.shape)
print('x_test shape:', processed_x_test.shape)

#from tensorflow.keras.engine.topology import Layer
from tensorflow.keras.layers import Layer    

import keras.backend
from keras.models import Sequential, load_model
from keras.layers import CuDNNGRU, Dense, Conv1D, MaxPooling1D
from keras.layers import Dropout, GlobalMaxPooling1D, BatchNormalization, LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

import os

embeddings_index = {}
f = open(os.path.join('/content/drive/MyDrive/Colab_Notebooks', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 100
k = 0
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        k += 1
        embedding_matrix[i] = embedding_vector

from scipy import ndimage, misc
import matplotlib.pyplot as plt



import tensorflow as tf
import tensorflow_io as tfio


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a = K.expand_dims(a)
        #print(type(a))
        #print("Shape is : ",a.get_shape)
        #a=tf.reshape(a,[1,None,415,120])
        a = K.expand_dims(a)
        #print("Shape is : ",a.get_shape)
        print(a.shape)


        a=tfio.experimental.filter.laplacian(a,5)
        
        a=K.squeeze(a,3)



        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        x = K.expand_dims(x)
        
        x=tfio.experimental.filter.laplacian(x,5)
        
        #a = tf.reshape(a,[None,415,120])
        x =K.squeeze(x,3)

        x = K.expand_dims(x)
        
        x=tfio.experimental.filter.laplacian(x,5)
        
        #a = tf.reshape(a,[None,415,120])
        x =K.squeeze(x,3)

        x = K.expand_dims(x)
        
        #x=tfio.experimental.filter.laplacian(x,5)
        
        #a = tf.reshape(a,[None,415,120])
        x =K.squeeze(x,3)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, concatenate
from keras.layers.merge import concatenate
from keras.models import Model

import keras.backend
from keras.models import Sequential, load_model
from keras.layers import CuDNNGRU, Dense, Conv1D, MaxPooling1D
from keras.layers import Dropout, GlobalMaxPooling1D, BatchNormalization, LSTM
from keras.layers import Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Nadam
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

from keras import backend as K


def get_model():
    review_input = Input(shape=(max_len,), dtype='int32')
    #review_input_post = Input(shape=(max_len,), dtype='int32')
    
    
    x1 = Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(review_input)
    x1 = Bidirectional(LSTM(60, return_sequences=True))(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Attention(max_len)(x1)
    """
    x2 = Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(review_input_post)
    x2 = Bidirectional(LSTM(60, return_sequences=True))(x2)
    x2 = Dropout(0.3)(x2)
    x2 = Attention(max_len)(x2)
    """
    x = x1 # concatenate([x1, x2])
    x = Dense(50, activation='relu')(x)
    x= Dropout(0.2)(x)
    x = BatchNormalization()(x)
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[review_input], outputs=preds)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = get_model()

model.summary()


"""
review_input = Input(shape=(max_len,), dtype='int32')
review_input_post = Input(shape=(max_len,), dtype='int32')
    #comment_input=Input(dtype='int32')
    #comment_input_post=Input(dtype='int32')
    #embedding_matrix=torch.Tensor(embedding_matrix)
model=Sequential()
model.add(Embedding(vocab_size + 1, embedding_dim, weights=[torch.Tensor(embedding_matrix)], input_length=max_len, trainable=True))#(comment_input)
model.add(Bidirectional(LSTM(60, return_sequences=True)))#(x1)
model.add(Dropout(0.3))#(x1)
model.add(Attention(embedding_dim))#(x1)

model.add(Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))#(comment_input_post)
model.add(Bidirectional(LSTM(60, return_sequences=True)))#(x2)
model.add(Dropout(0.3))#(x2)
model.add(Attention(input_length))#(x2)

    #x = concatenate([x1, x2])
model.add(Dense(50, activation='relu'))#(x)
model.add(Dropout(0.2))#(x)
model.add(BatchNormalization())#(x)
preds = Dense(1, activation='sigmoid')#(x)
model = Model(inputs=[review_input, review_input_post], outputs=preds)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

"""


from keras.callbacks import ModelCheckpoint

# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10)

history = model.fit(processed_x_train,y_train, validation_data=(processed_x_val,y_val), epochs=10,batch_size=32)#,callbacks=[early_stopping_monitor])
                    
model.evaluate(processed_x_test,y_test)





In [ ]:
print(processed_x_train[0])
print(processed_pre_x_train[0])


In [ ]:
classified=model.predict(processed_x_test)
classified=np.concatenate(classified,axis=0)
misclassified=(np.round(classified)!= y_test )
#print(misclassified)
dfm=pd.DataFrame()
dfm['m']=misclassified
#print(dfm)
i=dfm.index[dfm['m'] == True].tolist()
print(i)
print(len(i))
print(len(processed_x_test))
for j in i:
    print(df.iloc[j,:])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()